<a href="https://colab.research.google.com/github/EricRoh-kr/st9_youtubevid/blob/main/st9_youtubevid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# 필요한 라이브러리 설치 및 로드
import subprocess, sys, os, zipfile, shutil, glob, re
from datetime import datetime
from pytz import timezone
from IPython.display import display, FileLink, clear_output
import ipywidgets as widgets
from urllib.parse import urlparse, parse_qs

# Colab 감지
try:
    from google.colab import files
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

IS_RUNNING = False

#햠수들...
def install_ytdlp_ffmpeg():
    try:
        subprocess.check_call(
            [sys.executable, "-m", "pip", "install", "-U", "yt-dlp"],
            stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL
        )
    except subprocess.CalledProcessError:
        return False

    if IN_COLAB:
        subprocess.run(["apt-get", "install", "-y", "ffmpeg"], stdout=subprocess.DEVNULL)

    global YoutubeDL
    from yt_dlp import YoutubeDL

    return True

def make_progress_hook(out, idx, total, url):
    def hook(d):
        status = d.get('status')
        if status == 'downloading':
            percent = d.get('_percent_str', '').strip()
            speed   = d.get('_speed_str', '')
            eta     = d.get('_eta_str', '')
            out.clear_output(wait=True)
            with out:
                print(f"⬇️ [{idx}/{total}] 다운로드 중: {url}")
                print(f"   진행률: {percent} | 속도: {speed} | 남은 시간: {eta}")
        elif status == 'finished':
            with out:
                print(f"✅ [{idx}/{total}] 다운로드 완료: {url}\n")
    return hook

def normalize_url(url):
    url = url.strip()
    parsed = urlparse(url)

    # (1) 공유링크 (youtu.be/VIDEO_ID)
    if parsed.netloc in ["youtu.be"]:
        video_id = parsed.path.lstrip('/')
        return f"https://youtu.be/{video_id}"

    # (2) watch 링크 (youtube.com/watch?v=VIDEO_ID)
    if parsed.netloc in ["www.youtube.com", "youtube.com"]:
        qs = parse_qs(parsed.query)
        if 'v' in qs:
            return f"https://www.youtube.com/watch?v={qs['v'][0]}"

    return url

def download_videos(urls, folder, mode, out):
    total = len(urls)
    for i, u in enumerate(urls, 1):
        clean_url = normalize_url(u)
        hook = make_progress_hook(out, i, total, clean_url)
        probe_opts = {'quiet': True, 'no_warnings': True, 'skip_download': True}
        with YoutubeDL(probe_opts) as ydl:
            info = ydl.extract_info(clean_url, download=False)
        w, h = info.get('width', 0), info.get('height', 0)

        if mode == "기본 다운로드":
            if w >= h:
                # 가로 영상: 높이 기준 1080 이하
                vchain = (
                    "bestvideo[vcodec^=avc1][ext=mp4][height<=1080]"
                    "/bestvideo[ext=mp4][height<=1080]"
                    "/bestvideo[height<=1080]"
                )
            else:
                # 세로 영상: 너비 기준 1080 이하
                vchain = (
                    "bestvideo[vcodec^=avc1][ext=mp4][width<=1080]"
                    "/bestvideo[ext=mp4][width<=1080]"
                    "/bestvideo[width<=1080]"
                )
        else:
            # 최고 화질 모드: 해상도 제한 없이 avc1 → mp4 → 아무거나
            vchain = (
                "bestvideo[vcodec^=avc1][ext=mp4]"
                "/bestvideo[ext=mp4]"
                "/bestvideo"
            )

        # 오디오: m4a 우선, 없으면 아무거나
        achain = "bestaudio[ext=m4a]/bestaudio"

        # 최종 포맷: 비디오+오디오, 그래도 실패하면 단일 best(mp4) → best
        fmt = f"({vchain})+({achain})/(best[ext=mp4]/best)"

        ydl_opts = {
            'cookies' : 'www.youtube.com_cookies.txt',
            'format': fmt,
            'merge_output_format': 'mp4',
            'outtmpl': f'{folder}/%(title)s.%(ext)s',
            'noplaylist': True,
            'progress_hooks': [hook],
            'quiet': True,
            'no_warnings': True
        }

        with out:
            print(f"\n▶️ [{i}/{total}] 다운로드 시작: {clean_url}")

        with YoutubeDL(ydl_opts) as ydl:
            ydl.download([clean_url])


        # --- 호환성 체크 및 변환 ---
        downloaded_files = glob.glob(f"{folder}/*.mp4")
        if not downloaded_files:
            continue  # 혹시 다운로드 실패 시 skip
        downloaded_file = max(downloaded_files, key=os.path.getctime)  # 최신 파일 추출

        # 비디오 코덱 확인
        video_codec = subprocess.run([
            "ffprobe", "-v", "error", "-select_streams", "v:0",
            "-show_entries", "stream=codec_name", "-of", "csv=p=0", downloaded_file
        ], capture_output=True, text=True).stdout.strip()

        # 오디오 코덱 확인
        audio_codec = subprocess.run([
            "ffprobe", "-v", "error", "-select_streams", "a:0",
            "-show_entries", "stream=codec_name", "-of", "csv=p=0", downloaded_file
        ], capture_output=True, text=True).stdout.strip()

        # PPT 호환 여부 판단 (H.264 + AAC)
        if not (video_codec.startswith("h264") and audio_codec.startswith("aac")):
            print(f"⚠ PPT 호환 안 됨 → H.264 + AAC 변환 중: {os.path.basename(downloaded_file)}")
            converted_file = downloaded_file.replace(".mp4", "_ppt.mp4")

            # ★ 변경된 부분: 진행률 표시 함수 호출
            convert_with_progress(downloaded_file, converted_file)

            os.remove(downloaded_file)

def convert_with_progress(input_file, output_file):
    progress_bar = widgets.IntProgress(min=0, max=100, value=0)
    progress_label = widgets.Label(value="인코딩 진행률: 0%")
    with out:
        clear_output(wait=True)
        display(progress_bar, progress_label)

    cmd = [
        "ffmpeg", "-y", "-i", input_file,
        "-c:v", "libx264", "-pix_fmt", "yuv420p",
        "-c:a", "aac", "-b:a", "192k",
        output_file
    ]
    process = subprocess.Popen(cmd, stderr=subprocess.PIPE, universal_newlines=True)

    duration = None
    for line in process.stderr:
        if duration is None:
            match = re.search(r"Duration: (\d+):(\d+):(\d+\.\d+)", line)
            if match:
                h, m, s = match.groups()
                duration = int(h) * 3600 + int(m) * 60 + float(s)
        time_match = re.search(r"time=(\d+):(\d+):(\d+\.\d+)", line)
        if time_match and duration:
            h, m, s = time_match.groups()
            elapsed = int(h) * 3600 + int(m) * 60 + float(s)
            progress = (elapsed / duration) * 100
            progress_bar.value = int(progress)
            progress_label.value = f"인코딩 진행률: {progress:.1f}%"

    process.wait()
    progress_bar.value = 100
    progress_label.value = "인코딩 완료: 100%"
    return process.returncode == 0

def make_zip_and_cleanup(folder, out):
    for p in glob.glob("*_유튜브다운로드.zip"):
        try: os.remove(p)
        except: pass

    now = datetime.now(timezone('Asia/Seoul')).strftime("%m%d_%H%M")
    zip_name = f"{now}_유튜브다운로드.zip"
    with out: print("💾 파일 압축 중...")
    with zipfile.ZipFile(zip_name, "w", zipfile.ZIP_DEFLATED) as zf:
        for root, _, files in os.walk(folder):
            for f in files:
                full = os.path.join(root, f)
                zf.write(full, os.path.relpath(full, folder))
    return zip_name

def on_click(b):
    global IS_RUNNING
    if IS_RUNNING: return
    IS_RUNNING = True
    button.disabled = True
    out.clear_output()

    with out:
        urls = [u.strip() for u in urls_input.value.splitlines() if u.strip()]
        if not urls:
            print("❌ 링크를 입력해주세요."); IS_RUNNING=False; button.disabled=False; return

        print("🚀 yt-dlp & ffmpeg 설치 확인...")
        if not install_ytdlp_ffmpeg():
            print("❌ 설치 실패"); IS_RUNNING=False; button.disabled=False; return

        folder = "영상저장함"
        if os.path.exists(folder): shutil.rmtree(folder)
        os.makedirs(folder)

        download_videos(urls, folder, quality_choice.value, out)
        zip_name = make_zip_and_cleanup(folder, out)
        print(f"✅ ZIP 완성: {zip_name}")

        if IN_COLAB: files.download(zip_name)
        else:
            print("👉 링크 클릭 후 다운로드:")
            display(FileLink(zip_name))

    IS_RUNNING = False
    button.disabled = False

# UI
urls_input = widgets.Textarea(
    placeholder="유튜브 링크를 한 줄씩 입력하세요",
    layout=widgets.Layout(width="90%", height="120px")
)
quality_choice = widgets.RadioButtons(
    options=["기본 다운로드", "(⚠공사중) 최고 화질 다운로드"],
    description="화질 모드:", style={'description_width': 'initial'}
)
quality_description = widgets.HTML("""
<div style="border:1px solid #ccc; padding:10px; font-size:14px; line-height:1.5;">
<b>기본 다운로드 : </b> 1080p 화질로 다운로드하며, 속도가 비교적 빠릅니다.<br>
<b>최고 화질 다운로드 : </b> (현재 공사 중입니다) 제공 가능한 최고 화질로 다운로드한 뒤, 필요한 변환 과정을 거칩니다. 속도가 느리고 용량이 커질 수 있습니다.<br><br>
하단의 <b>"▶ 영상 다운로드"</b> 버튼을 누르게 되면 요청 링크들의 영상을 다운, 취합하여 압축 파일로 다운받게 됩니다.
</div>
""")
button = widgets.Button(
    description="▶ 영상 다운로드", button_style="success",
    layout=widgets.Layout(width="200px", height="40px")
)
out = widgets.Output(layout=widgets.Layout(width="90%", border="1px solid #ddd", padding="10px"))

button.on_click(on_click)

clear_output(wait=True)
display(widgets.VBox([
    widgets.HTML("<h2>🎬 전략 9팀 YouTube 다운로더</h2>"),
    urls_input, quality_choice, quality_description,
    button, widgets.HTML("<hr><h3>진행 상황</h3>"), out
]))